# Final Project: MeetUp DataFrames.

In [1]:
reset -fs

In [180]:
import json
from pyspark.sql.functions import explode
from pyspark.sql.functions import col
from datetime import datetime

In [63]:
#df = spark.read.json("s3a://dsci6007-firehose-final-project/meet_up2017/02/13/22/dsci6007_firehose_meet_up-1-2017-02-13-22-39-18-15eb685e-17b3-4a6a-aefb-c7309eed423c")

In [181]:
df = spark.read.json("s3a://dsci6007-firehose-final-project/meet_up2017/02/*/*/*")

In [182]:
results_df_1 = df.select('results')

In [4]:
df.select('results.id', 'results.city').show()

+--------------------+--------------------+
|                  id|                city|
+--------------------+--------------------+
|[18590249, 185903...|[Mountain View, S...|
+--------------------+--------------------+



In [65]:
results_df_1.printSchema()

root
 |-- results: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- category: struct (nullable = true)
 |    |    |    |-- id: long (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- shortname: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- created: long (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- group_photo: struct (nullable = true)
 |    |    |    |-- base_url: string (nullable = true)
 |    |    |    |-- highres_link: string (nullable = true)
 |    |    |    |-- photo_id: long (nullable = true)
 |    |    |    |-- photo_link: string (nullable = true)
 |    |    |    |-- thumb_link: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- join_mode: string (nullable = true)
 |    |    |-- lat: double (null

In [8]:
#results_df.select(results_df['results'][0]).show()

In [7]:
results_df_1.select(results_df_1['results']).show()

+--------------------+
|             results|
+--------------------+
|[[[6,education/le...|
+--------------------+



In [183]:
results_df_2 = results_df_1.select(results_df_1['results'])

## Get `category`.

In [5]:
results_df_2.select(explode(results_df_2['results']['category'])).distinct().show()

+--------------------+
|                 col|
+--------------------+
|[16,language/ethn...|
|[32,sports/recrea...|
|[24,paranormal,pa...|
|                null|
|[15,hobbies/craft...|
|[6,education/lear...|
|    [21,music,music]|
|      [12,LGBT,lgbt]|
|[17,alternative l...|
|    [11,games,games]|
|[10,food/drink,fo...|
|[29,sci-fi/fantas...|
|[31,socializing,s...|
|[26,pets/animals,...|
|[13,movements/pol...|
|[3,cars/motorcycl...|
|[20,movies/film,m...|
|[2,career/busines...|
|[8,fashion/beauty...|
| [9,fitness,fitness]|
+--------------------+
only showing top 20 rows



In [184]:
results_df_2_1 = results_df_2.select(explode(results_df_2['results']['category'])).distinct()

In [185]:
results_df_2_1_no_unique = results_df_2.select(explode(results_df_2['results']['category']))

In [7]:
results_df_2_1.select(results_df_2_1.col['id'].alias('category_id'), results_df_2_1.col['shortname'].alias('category_shortname'), results_df_2_1.col['name'].alias('category_name')).where(results_df_2_1.col['id'].isNotNull()).show()

+-----------+-------------------+--------------------+
|category_id| category_shortname|       category_name|
+-----------+-------------------+--------------------+
|         16|           language|language/ethnic i...|
|         32|  sports-recreation|   sports/recreation|
|         24|         paranormal|          paranormal|
|         15|     hobbies-crafts|      hobbies/crafts|
|          6| education-learning|  education/learning|
|         21|              music|               music|
|         12|               lgbt|                LGBT|
|         17|          lifestyle|alternative lifes...|
|         11|              games|               games|
|         10|         food-drink|          food/drink|
|         29|     sci-fi-fantasy|      sci-fi/fantasy|
|         31|        socializing|         socializing|
|         26|       pets-animals|        pets/animals|
|         13|government-politics|  movements/politics|
|          3|   cars-motorcycles|    cars/motorcycles|
|         

In [186]:
category_df = results_df_2_1.select(results_df_2_1.col['id'].alias('category_id'), results_df_2_1.col['shortname'].alias('category_shortname'), results_df_2_1.col['name'].alias('category_name')).where(results_df_2_1.col['id'].isNotNull())

In [187]:
category_no_unique_df = results_df_2_1_no_unique.select(results_df_2_1_no_unique.col['id'].alias('category_id'), results_df_2_1_no_unique.col['shortname'].alias('category_shortname'), results_df_2_1_no_unique.col['name'].alias('category_name')).where(results_df_2_1_no_unique.col['id'].isNotNull())

In [188]:
category_df.show()

+-----------+-------------------+--------------------+
|category_id| category_shortname|       category_name|
+-----------+-------------------+--------------------+
|         16|           language|language/ethnic i...|
|         32|  sports-recreation|   sports/recreation|
|         24|         paranormal|          paranormal|
|         15|     hobbies-crafts|      hobbies/crafts|
|          6| education-learning|  education/learning|
|         21|              music|               music|
|         12|               lgbt|                LGBT|
|         17|          lifestyle|alternative lifes...|
|         11|              games|               games|
|         10|         food-drink|          food/drink|
|         29|     sci-fi-fantasy|      sci-fi/fantasy|
|         31|        socializing|         socializing|
|         26|       pets-animals|        pets/animals|
|         13|government-politics|  movements/politics|
|          3|   cars-motorcycles|    cars/motorcycles|
|         

In [192]:
category_df.write.save('s3a://dsci6007-final-project-3nf/category_table/category_table.csv/', format="csv")

In [17]:
category_no_unique_df.groupBy('category_shortname').count().sort(col("count").desc()).show()

+--------------------+-----+
|  category_shortname|count|
+--------------------+-----+
|                tech| 9754|
|     career-business| 9288|
|         socializing| 5815|
|            language| 5390|
|  outdoors-adventure| 3816|
|          food-drink| 2973|
|               music| 2881|
|    health-wellbeing| 2537|
|             fitness| 2177|
|  education-learning| 2136|
|        pets-animals| 2121|
|new-age-spirituality| 2005|
|         movies-film| 1913|
|             dancing| 1847|
|        arts-culture| 1748|
| government-politics| 1696|
|                lgbt| 1564|
|             singles| 1352|
|               games| 1347|
|community-environ...| 1314|
+--------------------+-----+
only showing top 20 rows



In [193]:
category_count_df = category_no_unique_df.groupBy('category_shortname').count().sort(col("count").desc())

### Export to Pandas.

In [194]:
df_categories = category_count_df.limit(20).toPandas()

In [195]:
df_categories.head()

,category_shortname,count
0,tech,10729
1,career-business,10341
2,socializing,6478
3,language,6131
4,outdoors-adventure,4284


In [196]:
df_categories.index = df_categories.index + 1

In [197]:
df_categories

,category_shortname,count
1,tech,10729
2,career-business,10341
3,socializing,6478
4,language,6131
5,outdoors-adventure,4284
6,food-drink,3324
7,music,3193
8,health-wellbeing,2693
9,pets-animals,2433
10,education-learning,2370


In [242]:
top_list_categories = df_categories.values.tolist()

In [199]:
top_list_categories

[[u'tech', 10729],
 [u'career-business', 10341],
 [u'socializing', 6478],
 [u'language', 6131],
 [u'outdoors-adventure', 4284],
 [u'food-drink', 3324],
 [u'music', 3193],
 [u'health-wellbeing', 2693],
 [u'pets-animals', 2433],
 [u'education-learning', 2370],
 [u'fitness', 2255],
 [u'new-age-spirituality', 2239],
 [u'movies-film', 2186],
 [u'dancing', 2081],
 [u'government-politics', 1969],
 [u'arts-culture', 1904],
 [u'lgbt', 1681],
 [u'games', 1542],
 [u'singles', 1508],
 [u'support', 1432]]

In [243]:
top_list_categories_u = [[str(item[0]),item[1]] for item in top_list_categories]

In [244]:
top_list_categories_u.insert(0, ['Category', 'Count'])

In [245]:
top_list_categories_u

[['Category', 'Count'],
 ['tech', 10729],
 ['career-business', 10341],
 ['socializing', 6478],
 ['language', 6131],
 ['outdoors-adventure', 4284],
 ['food-drink', 3324],
 ['music', 3193],
 ['health-wellbeing', 2693],
 ['pets-animals', 2433],
 ['education-learning', 2370],
 ['fitness', 2255],
 ['new-age-spirituality', 2239],
 ['movies-film', 2186],
 ['dancing', 2081],
 ['government-politics', 1969],
 ['arts-culture', 1904],
 ['lgbt', 1681],
 ['games', 1542],
 ['singles', 1508],
 ['support', 1432]]

In [246]:
top_list_categories_u = top_list_categories_u[:11]

### Saving dataframe as html format in the master node.

In [247]:
timestamp = '{}'.format(datetime.now())

In [253]:
html_1 = '<!DOCTYPE html><HTML><HEAD><TITLE>Top 20 MeetUp Categories</TITLE></HEAD> \
        <BODY>{}</BODY></HTML>'.format(df_categories.to_html().encode('utf-8'))

In [254]:
with open("spark-top-meetup-categories.html", "w") as outf:
    outf.write("<H1>Top 20 MeetUp Categories from Spark</H1>")
    outf.write("<H2>As of: " + timestamp + "</H2>")
    outf.write(html_1)

`scp -i ~/ssh/dsci6007_cpm.pem hadoop@ec2-52-202-107-142.compute-1.amazonaws.com:~/spark-top-meetup-categories.html /Users/carles/`

In [251]:
html_2 = '<html><head><title>Top 10 MeetUp Categories</title> \
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script> \
    <script type="text/javascript"> \
      google.charts.load(\'current\', {\'packages\':[\'corechart\']}); \
      google.charts.setOnLoadCallback(drawChart); \
      function drawChart() { var data = google.visualization.arrayToDataTable(%s);\
        var options = { \
          title: \'Top 10 Categories Breakdown\'}; \
        var chart = new google.visualization.PieChart(document.getElementById(\'piechart\')); \
        chart.draw(data, options); \
      } \
    </script> \
  </head> \
  <body> \
  <h1>Top 10 MeetUp Categories from Spark</h1> \
    <div id="piechart" style="width: 900px; height: 500px;"></div> \
</body> \
</html>'%(top_list_categories_u)

In [252]:
with open("spark-top-meetup-categories-pie.html", "w") as outf:
    outf.write(html_2)

`scp -i ~/ssh/dsci6007_cpm.pem hadoop@ec2-52-202-107-142.compute-1.amazonaws.com:~/spark-top-meetup-categories-pie.html /Users/carles/`

## Get `topics`.

In [26]:
results_df_2.select(explode(results_df_2['results']['topics'])).distinct().show()

+--------------------+
|                 col|
+--------------------+
|[[21184,European ...|
|[[849,Philosophy,...|
|[[1052,Goddess,go...|
|[[713,Dining Out,...|
|[[1193,Karaoke,ka...|
|[[286,Anime,anime...|
|[[206,Digital Pho...|
|[[498,Singles,sin...|
|[[242,Fitness,fit...|
|[[407,Moms,moms],...|
|[[407,Moms,moms],...|
|[[10454,Language ...|
|[[3006,Human Reso...|
|[[3336,Self-Impro...|
|[[459,HiFi,hifi],...|
|[[1322,Meditation...|
|[[10579,Technolog...|
|[[16,Sales,sales]...|
|[[1064,Python,pyt...|
|[[2702,Domestic V...|
+--------------------+
only showing top 20 rows



In [209]:
results_df_2_2 = results_df_2.select(explode(results_df_2['results']['topics'])).distinct()

In [210]:
results_df_2_2_no_unique = results_df_2.select(explode(results_df_2['results']['topics']))

In [31]:
results_df_2_2.select(explode(results_df_2_2.col)).show()

+--------------------+
|                 col|
+--------------------+
|[21184,European C...|
|[849,Philosophy,p...|
|[1768,Ethics,ethics]|
|[17957,Ontology,o...|
| [23311,Logic,logic]|
|[38404,Epistemolo...|
|[1052,Goddess,god...|
|[10290,Spirituali...|
|[17276,Women's Ci...|
|[17933,Women's Em...|
|[37479,Earth-Base...|
|[46860,New Moon,n...|
|[59894,Sacred Cer...|
|[62062,Sacred Fem...|
|[67184,Moon Circl...|
|[83103,new moon m...|
|[85708,New Moon R...|
|[97136,Goddess Sp...|
|[105750,Divine Gu...|
|[151485,The Divin...|
+--------------------+
only showing top 20 rows



In [211]:
results_df_2_2_1 = results_df_2_2.select(explode(results_df_2_2.col))

In [212]:
results_df_2_2_1_no_unique = results_df_2_2_no_unique.select(explode(results_df_2_2_no_unique.col))

In [35]:
results_df_2_2_1.select(results_df_2_2_1.col['id'].alias('topic_id'), results_df_2_2_1.col['name'].alias('topic_name'), results_df_2_2_1.col['urlkey'].alias('topic_urlkey')).where(results_df_2_2_1.col['id'].isNotNull()).show()

+--------+--------------------+--------------------+
|topic_id|          topic_name|        topic_urlkey|
+--------+--------------------+--------------------+
|   21184|    European Culture|    european-culture|
|     849|          Philosophy|          philosophy|
|    1768|              Ethics|              ethics|
|   17957|            Ontology|            ontology|
|   23311|               Logic|               logic|
|   38404|        Epistemology|        epistemology|
|    1052|             Goddess|             goddess|
|   10290|        Spirituality|        spirituality|
|   17276|      Women's Circle|       womens-circle|
|   17933| Women's Empowerment|  womens-empowerment|
|   37479|Earth-Based Spiri...|earth-based-spiri...|
|   46860|            New Moon|            new-moon|
|   59894|Sacred Ceremony a...|sacred-ceremony-a...|
|   62062|     Sacred Feminine|     sacred-feminine|
|   67184|        Moon Circles|        moon-circles|
|   83103| new moon meditation| new-moon-medit

In [213]:
topic_df = results_df_2_2_1.select(results_df_2_2_1.col['id'].alias('topic_id'), results_df_2_2_1.col['name'].alias('topic_name'), results_df_2_2_1.col['urlkey'].alias('topic_urlkey')).where(results_df_2_2_1.col['id'].isNotNull())

In [214]:
topic_no_unique_df = results_df_2_2_1_no_unique.select(results_df_2_2_1_no_unique.col['id'].alias('topic_id'), results_df_2_2_1_no_unique.col['name'].alias('topic_name'), results_df_2_2_1_no_unique.col['urlkey'].alias('topic_urlkey')).where(results_df_2_2_1_no_unique.col['id'].isNotNull())

In [215]:
topic_df.show()

+--------+--------------------+--------------------+
|topic_id|          topic_name|        topic_urlkey|
+--------+--------------------+--------------------+
|   21184|    European Culture|    european-culture|
|     849|          Philosophy|          philosophy|
|    1768|              Ethics|              ethics|
|   17957|            Ontology|            ontology|
|   23311|               Logic|               logic|
|   38404|        Epistemology|        epistemology|
|    1052|             Goddess|             goddess|
|   10290|        Spirituality|        spirituality|
|   17276|      Women's Circle|       womens-circle|
|   17933| Women's Empowerment|  womens-empowerment|
|   37479|Earth-Based Spiri...|earth-based-spiri...|
|   46860|            New Moon|            new-moon|
|   59894|Sacred Ceremony a...|sacred-ceremony-a...|
|   62062|     Sacred Feminine|     sacred-feminine|
|   67184|        Moon Circles|        moon-circles|
|   83103| new moon meditation| new-moon-medit

In [216]:
topic_df.write.save('s3a://dsci6007-final-project-3nf/topic_table/topic_table.csv/', format="csv")

In [217]:
topic_count_df = topic_no_unique_df.groupBy('topic_name').count().sort(col("count").desc())

### Export to Pandas.

In [218]:
df_topic = topic_count_df.limit(20).toPandas()

In [219]:
df_topic.head()

,topic_name,count
0,Social Networking,14509
1,Social,11181
2,Singles,8809
3,Entrepreneurship,8543
4,New In Town,8157


In [220]:
df_topic.index = df_topic.index + 1

In [221]:
df_topic

,topic_name,count
1,Social Networking,14509
2,Social,11181
3,Singles,8809
4,Entrepreneurship,8543
5,New In Town,8157
6,Nightlife,6541
7,Dining Out,6031
8,Fun Times,5465
9,Web Technology,5282
10,Self-Improvement,4662


In [234]:
top_list_topics = df_topic.values.tolist()

In [235]:
top_list_topics_u = [[str(item[0]),item[1]] for item in top_list_topics]

In [236]:
top_list_topics_u.insert(0, ['Topic', 'Count'])

In [237]:
top_list_topics_u = top_list_topics_u[:11]

### Saving dataframe as html format in the master node.

In [238]:
timestamp = '{}'.format(datetime.now())

In [227]:
html_1 = '<!DOCTYPE html><HTML><HEAD><TITLE>Top 20 MeetUp Topics</TITLE></HEAD> \
        <BODY>{}</BODY></HTML>'.format(df_topic.to_html().encode('utf-8'))

In [239]:
with open("spark-top-meetup-topics.html", "w") as outf:
    outf.write("<H1>Top 20 MeetUp Topics from Spark</H1>")
    outf.write("<H2>As of: " + timestamp + "</H2>")
    outf.write(html_1)

``scp -i ~/ssh/dsci6007_cpm.pem hadoop@ec2-52-202-107-142.compute-1.amazonaws.com:~/spark-top-meetup-topics.html /Users/carles/``

In [240]:
html_2 = '<html><head><title>Top 10 MeetUp Topics</title> \
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script> \
    <script type="text/javascript"> \
      google.charts.load(\'current\', {\'packages\':[\'corechart\']}); \
      google.charts.setOnLoadCallback(drawChart); \
      function drawChart() { var data = google.visualization.arrayToDataTable(%s);\
        var options = { \
          title: \'Top 10 Topics Breakdown\'}; \
        var chart = new google.visualization.PieChart(document.getElementById(\'piechart\')); \
        chart.draw(data, options); \
      } \
    </script> \
  </head> \
  <body> \
  <h1>Top 10 MeetUp Topics from Spark</h1> \
    <div id="piechart" style="width: 900px; height: 500px;"></div> \
</body> \
</html>'%(top_list_topics_u)

In [241]:
with open("spark-top-meetup-topics-pie.html", "w") as outf:
    outf.write(html_2)

`scp -i ~/ssh/dsci6007_cpm.pem hadoop@ec2-52-202-107-142.compute-1.amazonaws.com:~/spark-top-meetup-topics-pie.html /Users/carles/`

## Get `organizer`.

In [48]:
results_df_2.select(explode(results_df_2['results']['organizer'])).show()

+--------------------+
|                 col|
+--------------------+
|[126468982,John S...|
|[569876,Casey Hol...|
|[810537,Rachel,[h...|
|[2476565,Anja,[ht...|
|[93184502,Ben,[ht...|
|[159597932,Mauree...|
|[9719187,Sylvia,[...|
|[3672645,Johanna ...|
|[3136421,Kelly,[h...|
|[810537,Rachel,[h...|
|[2976074,Jeff W,[...|
|[3138653,lehua,[h...|
|[16301661,Brian W...|
|[2153979,Poki,[ht...|
|[357792,Misha,[ht...|
|[34068522,Investo...|
|[184888684,Peter ...|
|[34068522,Investo...|
|[3260745,Betsy Mo...|
|[2368765,Bosco So...|
+--------------------+
only showing top 20 rows



In [255]:
results_df_2_3_no_unique = results_df_2.select(explode(results_df_2['results']['organizer']))

In [256]:
results_df_2_3 = results_df_2_3_no_unique.distinct()

In [49]:
#results_df_2_3 = results_df_2.select(explode(results_df_2['results']['organizer']))

In [52]:
results_df_2_3.select(results_df_2_3.col['member_id'].alias('member_id'), results_df_2_3.col['name'].alias('organizer_name')).where(results_df_2_3.col['member_id'].isNotNull()).show()

+---------+--------------------+
|member_id|      organizer_name|
+---------+--------------------+
| 12683857|           StudioPFP|
|184514510|         Rima Rivera|
|172882602|            Galya 🐾|
|  8907492|          Punit Kaur|
|  1433351|       Jerry Schwarz|
|147459412| Laura Veeneman Bare|
|  4299035|            Margaret|
|  3329273| Aleksandar Gargenta|
|206896437|                  JR|
|207875505|     Sathish Chandra|
|184120393|        Marty Hubert|
|185000244|               Yasar|
|140809602|           Teal Swan|
|189415010|       Luanne Ludwig|
|  2368765|            Bosco So|
| 11071880| David Cao Mlogy.net|
| 23272311|       Kenny Burgher|
| 11008485|    Dora Maria Abreu|
| 47044212|               Susan|
|  8614660|Ajanta_BollywoodG...|
+---------+--------------------+
only showing top 20 rows



In [257]:
organizer_df = results_df_2_3.select(results_df_2_3.col['member_id'].alias('member_id'), results_df_2_3.col['name'].alias('organizer_name')).where(results_df_2_3.col['member_id'].isNotNull())

In [258]:
organizer_no_unique_df = results_df_2_3_no_unique.select(results_df_2_3_no_unique.col['member_id'].alias('member_id'), results_df_2_3_no_unique.col['name'].alias('organizer_name')).where(results_df_2_3_no_unique.col['member_id'].isNotNull())

In [259]:
organizer_df.show()

+---------+-------------------+
|member_id|     organizer_name|
+---------+-------------------+
| 12683857|          StudioPFP|
|184514510|        Rima Rivera|
|  8907492|         Punit Kaur|
|172882602|           Galya 🐾|
|  1433351|      Jerry Schwarz|
|  4299035|           Margaret|
|147459412|Laura Veeneman Bare|
|  3329273|Aleksandar Gargenta|
|140809602|          Teal Swan|
|189415010|      Luanne Ludwig|
|206896437|                 JR|
|207875505|    Sathish Chandra|
|184120393|       Marty Hubert|
|185000244|              Yasar|
|  2368765|           Bosco So|
| 11071880|David Cao Mlogy.net|
| 23272311|      Kenny Burgher|
| 11008485|   Dora Maria Abreu|
| 47044212|              Susan|
|207911016|      Alexis Sheehy|
+---------+-------------------+
only showing top 20 rows



In [260]:
organizer_df.write.save('s3a://dsci6007-final-project-3nf/organizer_table/organizer_table.csv/', format="csv")

https://aws.amazon.com/premiumsupport/knowledge-center/rds-import-data/

http://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/PostgreSQL.Procedural.Importing.html

In [261]:
organizer_count_df = organizer_no_unique_df.groupBy('organizer_name').count().sort(col("count").desc())

### Export to Pandas.

In [262]:
df_organizer = organizer_count_df.limit(20).toPandas()

In [263]:
df_organizer.head()

,organizer_name,count
0,chriss,1120
1,Rob,841
2,Toby Morning,840
3,Howard S.,840
4,Rachel,663


In [264]:
df_organizer.index = df_organizer.index + 1

In [265]:
df_organizer

,organizer_name,count
1,chriss,1120
2,Rob,841
3,Toby Morning,840
4,Howard S.,840
5,Rachel,663
6,Investor's Business Daily,560
7,Sailor Dave,560
8,David Politi,560
9,Igor Sinyak,560
10,Maureen,560


In [266]:
top_list_organizer = df_organizer.values.tolist()

In [267]:
top_list_organizer_u = [[str(item[0]),item[1]] for item in top_list_organizer]

In [268]:
top_list_organizer_u.insert(0, ['Organizer', 'Count'])

In [269]:
top_list_organizer_u = top_list_organizer_u[:11]

### Saving dataframe as html format in the master node.

In [270]:
timestamp = '{}'.format(datetime.now())

In [271]:
html_1 = '<!DOCTYPE html><HTML><HEAD><TITLE>Top 20 MeetUp Organizers</TITLE></HEAD> \
        <BODY>{}</BODY></HTML>'.format(df_organizer.to_html().encode('utf-8'))

In [272]:
with open("spark-top-meetup-organizers.html", "w") as outf:
    outf.write("<H1>Top 20 MeetUp Organizers from Spark</H1>")
    outf.write("<H2>As of: " + timestamp + "</H2>")
    outf.write(html_1)

`scp -i ~/ssh/dsci6007_cpm.pem hadoop@ec2-52-202-107-142.compute-1.amazonaws.com:~/spark-top-meetup-organizers.html /Users/carles/`

In [273]:
html_2 = '<html><head><title>Top 10 MeetUp Organizers</title> \
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script> \
    <script type="text/javascript"> \
      google.charts.load(\'current\', {\'packages\':[\'corechart\']}); \
      google.charts.setOnLoadCallback(drawChart); \
      function drawChart() { var data = google.visualization.arrayToDataTable(%s);\
        var options = { \
          title: \'Top 10 Organizers Breakdown\'}; \
        var chart = new google.visualization.PieChart(document.getElementById(\'piechart\')); \
        chart.draw(data, options); \
      } \
    </script> \
  </head> \
  <body> \
  <h1>Top 10 MeetUp Organizers from Spark</h1> \
    <div id="piechart" style="width: 900px; height: 500px;"></div> \
</body> \
</html>'%(top_list_organizer_u)

In [274]:
with open("spark-top-meetup-organizers-pie.html", "w") as outf:
    outf.write(html_2)

`scp -i ~/ssh/dsci6007_cpm.pem hadoop@ec2-52-202-107-142.compute-1.amazonaws.com:~/spark-top-meetup-organizers-pie.html /Users/carles/`